# CityBikes


In [ ]:
# Standard Library Imports
import os
import re
import json
from pprint import pprint

# Third-Party Imports
import folium
import requests
import pandas as pd
from haversine import haversine, Unit
from geopy.geocoders import Nominatim
from pandas import json_normalize


In [ ]:
# Constants
API_BASE_URL = "http://api.citybik.es/v2"
NETWORK_ID = 'velib'
NETWORKS_PATH = '../data/raw_networks.csv'
STATIONS_PATH = '../data/raw_stations.csv'

# Function to fetch JSON data from a URL
def fetch_json_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status()

        content_type = response.headers.get('content-type', '').lower()
        if 'application/json' in content_type:
            return response.json()
        else:
            print(f"Invalid content type: {content_type}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON response from {url}: {e}")
        return None
    

# Function to recursively collect all keys and their data types from a JSON-like object
def get_all_keys_with_types(obj, keys_with_types, parent_key=''):
    if isinstance(obj, dict):
        for key, value in obj.items():
            full_key = f"{parent_key}.{key}" if parent_key else key
            keys_with_types[full_key] = type(value).__name__
            get_all_keys_with_types(value, keys_with_types, full_key)
    elif isinstance(obj, list):
        for item in obj:
            get_all_keys_with_types(item, keys_with_types, parent_key)


# Function to process network data and save to CSV
def process_network_data(data):
    if data:
        all_keys_with_types = {}
        get_all_keys_with_types(data, all_keys_with_types)

        print("Keys and Types for {url}:")
        for key, data_type in sorted(all_keys_with_types.items()):
            print(f"{key}: {data_type}")

        total_networks = len(data.get("networks", []))
        print(f"Total number of networks: {total_networks}")

        # Flatten the JSON data
        flattened_data = []
        for network in data.get("networks", []):
            # Flatten each network entry and append to the list
            flattened_data.append(json_normalize(network))

        # Concatenate all flattened data into a single DataFrame
        df = pd.concat(flattened_data, ignore_index=True)

        # Save station network to a CSV file
        df.to_csv(NETWORKS_PATH, index=False)


# Function to process station data
def process_station_data(data):
    if data:
        all_keys_with_types = {}
        get_all_keys_with_types(data, all_keys_with_types)

        print("Keys and Types for {url}:")
        for key, data_type in sorted(all_keys_with_types.items()):
            print(f"{key}: {data_type}")

        # Extract station data from the JSON
        station_data = data.get("network", {}).get("stations", [])
        total_stations = len(station_data)
        print(f"Total number of stations: {total_stations}")

        # Flatten the JSON station data
        flattened_data = []
        for station in station_data:
            flattened_data.append(json_normalize(station))

        # Concatenate all flattened data into a single DataFrame
        df = pd.concat(flattened_data, ignore_index=True)

        # Save station data to a CSV file
        df.to_csv(STATIONS_PATH, index=False)


# Main function
def main():
    urls = [
        f"{API_BASE_URL}/networks",
        f"{API_BASE_URL}/networks/{NETWORK_ID}"
    ]

    for url in urls:
        print(f"Fetching data from {url}")
        data = fetch_json_data(url)

        if url == f"{API_BASE_URL}/networks":
            process_network_data(data)
        elif url == f"{API_BASE_URL}/networks/{NETWORK_ID}":
            process_station_data(data)

if __name__ == "__main__":
    main()


Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes).


In [ ]:
def get_stations_data(url):
    """Fetch data from API and return JSON."""
    try:
        with requests.Session() as session:
            response = session.get(url)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

def parse_stations_data(stations_data):
    """Parse stations data from JSON."""
    if not stations_data:
        return []

    return [
        {
            "name": station['name'],
            "latitude": station['latitude'],
            "longitude": station['longitude'],
            "free_bikes": station['free_bikes'],
            "empty_slots": station['empty_slots'],
            "total_bikes": station['empty_slots'] + station['free_bikes'],
            "usage_percentage": (
                station['empty_slots'] / (station['empty_slots'] + station['free_bikes'])
                if (station['empty_slots'] + station['free_bikes']) != 0 else 0
            )
        } for station in stations_data.get('network', {}).get('stations', [])
    ]

# Usage
stations_url = "http://api.citybik.es/v2/networks/velib"
stations_raw_data = get_stations_data(stations_url)
stations_parsed_data = parse_stations_data(stations_raw_data)

# Now stations_parsed_data contains the structured data



Put your parsed results into a DataFrame.


In [ ]:
# DataFrame creation
stations_df = pd.DataFrame(stations_parsed_data)


Filter stations based on API limits


In [ ]:
# Define broader latitude and longitude bounds
lat_min, lat_max = 48.85, 48.88  # Broader range for latitude to include both landmarks
lon_min, lon_max = 2.29, 2.35    # Broader range for longitude to include both landmarks


# Assuming df is your DataFrame with 'latitude' and 'longitude' columns

# Filter the DataFrame based on the defined bounds
filtered_df = stations_df[(stations_df['latitude'] >= lat_min) & (stations_df['latitude'] <= lat_max) &
                 (stations_df['longitude'] >= lon_min) & (stations_df['longitude'] <= lon_max)]

# Proceed with further processing on filtered_df


Filtered by Radius

In [ ]:
# Central point (Notre-Dame Cathedral)
center_lat, center_lon = 48.8606, 2.3376  # Coordinates of the Louvre


# Degree changes for 2 km radius
radius_lat = 0.015  # Approximate radius in degrees latitude
radius_lon = 0.020  # Approximate radius in degrees longitude

# Calculate new bounds
lat_min = center_lat - radius_lat
lat_max = center_lat + radius_lat
lon_min = center_lon - radius_lon
lon_max = center_lon + radius_lon

# Assuming df is your DataFrame with 'latitude' and 'longitude' columns

# Filter the DataFrame based on the defined bounds
filtered_df = stations_df[(stations_df['latitude'] >= lat_min) & (stations_df['latitude'] <= lat_max) &
                 (stations_df['longitude'] >= lon_min) & (stations_df['longitude'] <= lon_max)]

# Proceed with further processing on filtered_df


Save to CSV


In [ ]:
stations_df.to_csv('../data/stations.csv', index=False)


Descriptive Statistics


In [ ]:
print(stations_df.describe())


Checking data integrity and data types


In [ ]:
print(stations_df.info())


Data Cleaning. Checking for duplicates and Null Values.


In [ ]:
print(stations_df.duplicated().sum())
print(stations_df.isnull().sum())


## Visualization

In [ ]:
import folium

# Create a map centered around the approximate center of the expanded bounds
map_center = [(lat_min + lat_max) / 2, (lon_min + lon_max) / 2]
map_folium = folium.Map(location=map_center, zoom_start=14)

# Adding markers for each bike station in the filtered DataFrame
for index, row in stations_df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name'],
        icon=folium.Icon(icon='bicycle', color='blue')
    ).add_to(map_folium)

# Display the map
map_folium



### Initial Process

#### Interactice Search

In [ ]:
import requests

# Function to fetch and filter data
def search_bike_networks(query):
    url = "http://api.citybik.es/v2/networks"
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    networks = response.json()['networks']

    query = query.lower()
    filtered_networks = [
        network for network in networks
        if query in network['location']['city'].lower() or
           query in network['location']['country'].lower() or
           query in network['id'].lower()
    ]

    return filtered_networks

# Get user input and display results
query_input = input("Enter city, country code, or company ID: ").strip()

if query_input:
    search_results = search_bike_networks(query_input)

    # Print the total number of results
    total_results = len(search_results)
    print(f"Total number of results for '{query_input}': {total_results}")

    # Print the filtered results
    for network in search_results:
        print(f"Network ID: {network['id']}")
        location = f"{network['location']['city']}, {network['location']['country']}"
        print(f"Location: {location}")
        companies = ', '.join(network['company'])
        print(f"Company: {companies}")
        print()
else:
    print("No query provided.")

